<a href="https://colab.research.google.com/github/wa1575/Deep_Learning/blob/main/multiClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
keras.__version__

In [ ]:
import os, shutil

In [ ]:
# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_dir = './datasets/bob9/train'

# 소규모 데이터셋을 저장할 디렉터리
base_dir = './datasets/bob9_small'
if os.path.exists(base_dir):  # 반복적인 실행을 위해 디렉토리를 삭제합니다.
    shutil.rmtree(base_dir)   # 이 코드는 책에 포함되어 있지 않습니다.
os.mkdir(base_dir)

# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

clas = ['cats', 'dogs', 'ducks', 'eagles', 'foxs', 'hawks','hyenas',\
             'parrots', 'sparrows', 'wolves']

vtrain_cats_dir = []
vvalidation_cats_dir = []
vtest_cats_dir = []

for v in clas:
    # train dir
    train_cats_dir = os.path.join(train_dir, v)
    os.mkdir(train_cats_dir)

    # vali dir
    validation_cats_dir = os.path.join(validation_dir, v)
    os.mkdir(validation_cats_dir)

    # test dir
    test_cats_dir = os.path.join(test_dir, v)
    os.mkdir(test_cats_dir)
    
    vtrain_cats_dir.append(train_cats_dir)
    vvalidation_cats_dir.append(validation_cats_dir)
    vtest_cats_dir.append(test_cats_dir)
    
classifier = ['cat{}.jpg', 'dog{}.jpg', 'duck{}.jpg', 'eagle{}.jpg', 'fox{}.jpg', 'hawk{}.jpg','hyena{}.jpg',\
             'parrot{}.jpg', 'sparrow{}.jpg', 'wolf{}.jpg']

for v in range(10):
    # Train
    fnames = [classifier[v].format(i) for i in range(1, 101)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(vtrain_cats_dir[v], fname)
        shutil.copyfile(src, dst)

    # Vali
    fnames = [classifier[v].format(i) for i in range(101, 151)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(vvalidation_cats_dir[v], fname)
        shutil.copyfile(src, dst)

    # Test
    fnames = [classifier[v].format(i) for i in range(151, 201)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(vtest_cats_dir[v], fname)
        shutil.copyfile(src, dst)

In [ ]:
for v in range(10):
    print('훈련용', clas[v], '이미지 전체 개수:', len(os.listdir(vtrain_cats_dir[v])))
    print('검증용', clas[v], '이미지 전체 개수:', len(os.listdir(vvalidation_cats_dir[v])))
    print('테스트용', clas[v], '이미지 전체 개수:', len(os.listdir(vtest_cats_dir[v])))
    print()

훈련용 cats 이미지 전체 개수: 100
검증용 cats 이미지 전체 개수: 50
테스트용 cats 이미지 전체 개수: 50

훈련용 dogs 이미지 전체 개수: 100
검증용 dogs 이미지 전체 개수: 50
테스트용 dogs 이미지 전체 개수: 50

훈련용 ducks 이미지 전체 개수: 100
검증용 ducks 이미지 전체 개수: 50
테스트용 ducks 이미지 전체 개수: 50

훈련용 eagles 이미지 전체 개수: 100
검증용 eagles 이미지 전체 개수: 50
테스트용 eagles 이미지 전체 개수: 50

훈련용 foxs 이미지 전체 개수: 100
검증용 foxs 이미지 전체 개수: 50
테스트용 foxs 이미지 전체 개수: 50

훈련용 hawks 이미지 전체 개수: 100
검증용 hawks 이미지 전체 개수: 50
테스트용 hawks 이미지 전체 개수: 50

훈련용 hyenas 이미지 전체 개수: 100
검증용 hyenas 이미지 전체 개수: 50
테스트용 hyenas 이미지 전체 개수: 50

훈련용 parrots 이미지 전체 개수: 100
검증용 parrots 이미지 전체 개수: 50
테스트용 parrots 이미지 전체 개수: 50

훈련용 sparrows 이미지 전체 개수: 100
검증용 sparrows 이미지 전체 개수: 50
테스트용 sparrows 이미지 전체 개수: 50

훈련용 wolves 이미지 전체 개수: 100
검증용 wolves 이미지 전체 개수: 50
테스트용 wolves 이미지 전체 개수: 50

이제 200개의 훈련 이미지, 100개의 검증 이미지, 100개의 테스트 이미지가 준비되었습니다. 분할된 각 데이터는 클래마다 동일한 개수의 샘플을 포함합니다. 균형잡힌 이진 분류 문제이므로 정확도를 사용해 성공을 측정하겠습니다.

**네트워크 구성하기**
이전 예제에서 MNIST를 위해 간단한 컨브넷을 만들었습니다. 이제 컨브넷에 친숙해졌을 것입니다. 여기서 사용할 구조도 일반적으로 동일합니다. Conv2D(relu 활성화 함수 사용)와 MaxPooling2D 층을 번갈아 쌓은 컨브넷을 만들겠습니다.

이전보다 이미지가 크고 복잡한 문제이기 때문에 네트워크를 좀 더 크게 만들겠습니다. Conv2D + MaxPooling2D 단계를 하나 더 추가합니다. 이렇게 하면 네트워크의 용량을 늘리고 Flatten 층의 크기가 너무 커지지 않도록 특성 맵의 크기를 줄일 수 있습니다. 150 × 150 크기(임의로 선택한 것입니다)의 입력으로 시작해서 Flatten 층 이전에 7 × 7 크기의 특성 맵으로 줄어듭니다.

특성 맵의 깊이는 네트워크에서 점진적으로 증가하지만(32에서 128까지), 특성 맵의 크기는 감소합니다(150 × 150에서 7 × 7까지). 이는 거의 모든 컨브넷에서 볼 수 있는 전형적인 패턴입니다.

이진 분류 문제이므로 네트워크는 다중 분류를 위한 10 클래스(크기가 10인 Dense 층)와 softmax 활성화 함수로 끝납니다. 이 유닛은 각각의 클래스에 대한 확률을 인코딩할 것입니다.

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

층들을 거치면서 특성 맵의 차원이 어떻게 변하는지 살펴보겠습니다:

In [ ]:
model.summary()

In [ ]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

**데이터 전처리**
데이터는 네트워크에 주입되기 전에 부동 소수 타입의 텐서로 적절하게 전처리되어 있어야 합니다. 지금은 데이터가 JPEG 파일로 되어 있으므로 네트워크에 주입하려면 대략 다음 과정을 따릅니다.

사진 파일을 읽습니다.
JPEG 콘텐츠를 RGB 픽셀 값으로 디코딩합니다.
그다음 부동 소수 타입의 텐서로 변환합니다.
픽셀 값(0에서 255 사이)의 스케일을 [0, 1] 사이로 조정합니다(신경망은 작은 입력 값을 선호합니다).
좀 복잡하게 보일 수 있지만 다행히 케라스는 이런 단계를 자동으로 처리하는 유틸리티를 가지고 있습니다. 케라스는 keras.preprocessing.image에 이미지 처리를 위한 헬퍼 도구들을 가지고 있습니다. 특히 ImageDataGenerator 클래스는 디스크에 있는 이미지 파일을 전처리된 배치 텐서로 자동으로 바꾸어주는 파이썬 제너레이터를 만들어 줍니다. 이 클래스를 사용해 보겠습니다.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# 이미지 전처리 유틸리티 모듈
from keras.preprocessing import image
import matplotlib.pyplot as plt

# 증식할 이미지 선택합니다
img_path = './datasets/bob9_small/train/cats/cat1.jpg'

# 이미지를 읽고 크기를 변경합니다
img = image.load_img(img_path, target_size=(150, 150))

# (150, 150, 3) 크기의 넘파이 배열로 변환합니다
x = image.img_to_array(img)

# (1, 150, 150, 3) 크기로 변환합니다
x = x.reshape((1,) + x.shape)

# flow() 메서드는 랜덤하게 변환된 이미지의 배치를 생성합니다.
# 무한 반복되기 때문에 어느 지점에서 중지해야 합니다!
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# 모든 이미지를 1/255로 스케일을 조정합니다
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 150 × 150 크기로 바꿉니다
        target_size=(150, 150),
        batch_size=10,
        # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=10,
        class_mode='categorical')

이 제너레이터의 출력 하나를 살펴보죠. 이 출력은 150 × 150 RGB 이미지의 배치((20, 150, 150, 3) 크기)와 이진 레이블의 배치((20,) 크기)입니다. 각 배치에는 20개의 샘플(배치 크기)이 있습니다. 제너레이터는 이 배치를 무한정 만들어 냅니다. 타깃 폴더에 있는 이미지를 끝없이 반복합니다. 따라서 반복 루프안의 어디에선가 break 문을 사용해야 합니다.

In [ ]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

제너레이터를 사용한 데이터에 모델을 훈련시켜 보겠습니다. fit_generator 메서드는 fit 메서드와 동일하되 데이터 제너레이터를 사용할 수 있습니다. 이 메서드는 첫 번째 매개변수로 입력과 타깃의 배치를 끝없이 반환하는 파이썬 제너레이터를 기대합니다. 데이터가 끝없이 생성되기 때문에 케라스 모델에 하나의 에포크를 정의하기 위해 제너레이터로부터 얼마나 많은 샘플을 뽑을 것인지 알려 주어야 합니다. steps_per_epoch 매개변수에서 이를 설정합니다. 제너레이터로부터 steps_per_epoch 개의 배치만큼 뽑은 다음, 즉 steps_per_epoch 횟수만큼 경사 하강법 단계를 실행한 다음에 훈련 프로세스는 다음 에포크로 넘어갑니다. 여기서는 20개의 샘플이 하나의 배치이므로 2,000개의 샘플을 모두 처리할 때까지 100개의 배치를 뽑을 것입니다.

fit_generator를 사용할 때 fit 메서드와 마찬가지로 validation_data 매개변수를 전달할 수 있습니다. 이 매개변수에는 데이터 제너레이터도 가능하지만 넘파이 배열의 튜플도 가능합니다. validation_data로 제너레이터를 전달하면 검증 데이터의 배치를 끝없이 반환합니다. 따라서 검증 데이터 제너레이터에서 얼마나 많은 배치를 추출하여 평가할지 validation_steps 매개변수에 지정해야 합니다.

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

훈련이 끝나면 항상 모델을 저장하는 것이 좋은 습관입니다:

In [ ]:
model.save('cats_and_dogs_small_1.h5')

훈련 데이터와 검증 데이터에 대한 모델의 손실과 정확도를 그래프로 나타내 보겠습니다:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()